In [37]:
# Dependencies
import pandas as pd
import numpy as np
import time
from citipy import citipy
import requests
from config import weather_api_key

# API
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key


In [38]:
# 3. Create a new list of 2000 random coordinates

lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)

# Add the latitudes and longitudes to a list
coordinates = list(zip(lats, lngs))
# coordinates

In [39]:
# 4. Get the nearest cities with citipy
cities = []

# ID the nearest city for each lat/long combination
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list

    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)


777

In [40]:
# 5. Perform the API call with the OpenWeatherApp

city_data = []

# Print the beginning of the logging.

print("----------------------------------")
print("     Beginning Data Retrieval     ")
print("----------------------------------")


# Create counters

record_count = 1
set_count = 1

# Loop through all of the cities in the list
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(10)

    # Create endpoint URL with each city
    city_url = url + "&q=" + city.replace(" ", "+")

    # Log the URL, record, and set numbers and the city

    print(f"Processing Record {record_count} of Set {set_count} | {city}")

    # Add 1 to the record count

    record_count += 1

# 6. Retreive the info from the API call - country, lat/long, max temp, humidity, cloudiness, wind speed, description
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        
        # Description is located in a dictionary within a list - what a pain in the ass
        city_description = [i["description"] for i in city_weather["weather"]][0].capitalize()



        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_description
                          })

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------------")
print("      Data Retrieval Complete      ")
print("-----------------------------------")


----------------------------------
     Beginning Data Retrieval     
----------------------------------
Processing Record 1 of Set 1 | tsihombe
City not found. Skipping...
Processing Record 2 of Set 1 | toliary
City not found. Skipping...
Processing Record 3 of Set 1 | tuktoyaktuk
Processing Record 4 of Set 1 | jamestown
Processing Record 5 of Set 1 | baykit
Processing Record 6 of Set 1 | busselton
Processing Record 7 of Set 1 | lewistown
Processing Record 8 of Set 1 | chokurdakh
Processing Record 9 of Set 1 | zhanaozen
Processing Record 10 of Set 1 | barrow
Processing Record 11 of Set 1 | kebemer
City not found. Skipping...
Processing Record 12 of Set 1 | bengkulu
Processing Record 13 of Set 1 | vaini
Processing Record 14 of Set 1 | raudeberg
Processing Record 15 of Set 1 | bethel
Processing Record 16 of Set 1 | cabo rojo
Processing Record 17 of Set 1 | panama city
Processing Record 18 of Set 1 | georgetown
Processing Record 19 of Set 1 | jaisalmer
Processing Record 20 of Set 1 | alb

In [41]:
# Add the array of dictionaries a new DF

city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)


,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Tuktoyaktuk,CA,69.4541,-133.0374,50.00,81,100,10.36,Overcast clouds
1,Jamestown,US,42.0970,-79.2353,78.53,55,40,9.22,Scattered clouds
2,Baykit,RU,61.6700,96.3700,51.69,93,84,2.30,Broken clouds
3,Busselton,AU,-33.6500,115.3333,47.79,85,0,9.62,Clear sky
4,Lewistown,US,40.5992,-77.5714,86.34,65,40,6.91,Scattered clouds
5,Chokurdakh,RU,70.6333,147.9167,51.93,51,11,18.14,Few clouds
6,Zhanaozen,KZ,43.3000,52.8000,78.04,36,1,12.39,Clear sky
7,Barrow,US,71.2906,-156.7887,51.82,76,100,25.32,Light rain
8,Bengkulu,ID,-3.8004,102.2655,73.63,88,100,12.93,Moderate rain
9,Vaini,TO,-21.2000,-175.2000,77.16,94,75,12.66,Broken clouds


In [42]:
# 8. Export the data to the Database folder
output_file = "WeatherPy_Database.csv"

city_data_df.to_csv(output_file, index_label="City_ID")
